# Extracting the total working population by districts in India

The Census 2011 India consists of detailed data on the working population in India.
Following data processing is done to get the desired data:

1. Ingest one by one all the raw consisting the raw data
2. Process each file to get the desired data
3. Write the desired data for each state and one consolidated file for the whole country into CSV files.

In [2]:
import pandas as pd
import os

The **inFolder** consists of all the raw files and the **outFolder** consists of all the files with desired data.

In [217]:
outFolder = "~/Working Population/Out/"
inFolder = "~/In/"

In [218]:
files = os.listdir(inFolder)

The below cell:

1. loops over each raw file
2. processes them to extract the required data
3. writes a state CSV file
4. consolidates all the state's data
5. writes the consolidated data at last

In [231]:
Final_df = pd.DataFrame()
for file in files:
    df = pd.read_excel(inFolder+file)
    df.drop([0,1,2,3,4,6],axis=0,inplace=True)
    df.columns = df.iloc[0]
    df.drop([5],inplace=True)
    State = df[5].iloc[0][8:][:-5]
    df = df[[5,6,7,11,14,17,20]]
    df[['State','Name']] = df[5].str.split(' - ',expand=True,n=1)
    df = df[df.State == 'District']
    df = df[df[6] == 'Total']
    df = df[df[7] == 'Total']
    df['Working Pop'] = df[11]+df[14]+df[17]+df[20]
    df['State'] = State
    df = df.drop([5,6,7],axis=1)
    df.columns = ['Total Main Worker','Total Marginal Workers (<3 months)','Total Marginal Workers (3 - 6 months)','Total Workers Seeking/Available for Work','State','District','Total Working Population']
    df.reset_index(inplace=True,drop=True)
    df.District = df.District.apply(lambda x: x[:-5])
    df = df.reindex(columns=['State','District','Total Main Worker','Total Marginal Workers (<3 months)','Total Marginal Workers (3 - 6 months)','Total Workers Seeking/Available for Work','Total Working Population'])
    df.to_csv(outFolder+State+'.csv',index=False)
    Final_df = pd.concat([Final_df,df])
Final_df.to_excel(outFolder+'Working Population.xlsx',index=False)